<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h1>
Old GDP Revisions Datasets
</h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
This <span style="color: rgb(0, 65, 75);">jupyter notebook</span> documents step-by-step the <b>construction of old datasets</b> for the project <b>'Revisions and Biases in Preliminary GDP Estimates in Peru'</b>.

...
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. </a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Data cleaning</a>
    <br>
    <a href="#2-1" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Extracting tables and data cleanup.</a>
    <br>
    <a href="#2-2-1" style="color: #94FFD8; font-size: 14px; margin-left: 40px;">
        2.2.1. Table 1. Extraction and cleaning of data from tables on monthly real GDP growth rates.</a>
    <br>
    <a href="#2-2-2" style="color: #94FFD8; font-size: 14px; margin-left: 40px;">
        2.2.2. Table 2. Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.</a>
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">3. Real-time data of Peru's GDP growth rates</a>
    <br>
    <a href="#3-1" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.1. Annual vintages concatenation.</a>
    <br>
    <a href="#3-2" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.2. Quarterly vintages concatenation.</a>
    <br>
    <a href="#3-3" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.3. Monthly vintages concatenation.</a>
    <br>
    <a href="#3-4" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.4. Loading SQL.</a>
    <br>
    <a href="#4" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">4. GDP final revision dataset</a>
    <br>
    <a href="#4-1" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        4.1. Annual revisions.</a>
    <br>
    <a href="#4-2" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        4.2. Quarterly revisions.</a>
    <br>
    <a href="#4-3" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        4.3. Monthly revisions.</a>
    <br>
    <a href="#5" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">5. Uploading data to SQL</a>
    <br>
    <a href="#5-1" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        5.1. Vintages.</a>
    <br>
    <a href="#5-2" style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        5.2. Revisions.</a>
</div>


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'charter'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# 1. Data cleaning
#-------------------------------------------------------------------------------------------------------------------------------

# 1.1. A brief documentation on issus in the table information of the PDFs

from PIL import Image  # Used for opening, manipulating, and saving image files.
import matplotlib.pyplot as plt  # Used for creating static, animated, and interactive visualizations.

# 1.2. Extracting tables and data cleanup


import pandas as pd  # For data manipulation and analysis
import unicodedata  # For manipulating Unicode data
import re  # For regular expressions operations
from datetime import datetime  # For working with dates and times
import locale  # For locale-specific formatting of numbers, dates, and currencies
import numpy as np
import unidecode

# 1.2.1. Table 1. Extraction and cleaning of data from tables on monthly real GDP growth rates.

import tabula  # Used to extract tables from PDF files into pandas DataFrames
from tkinter import Tk, messagebox, TOP, YES, NO  # Used for creating graphical user interfaces
from sqlalchemy import create_engine  # Used for connecting to and interacting with SQL databases

# 1.2.2. Table 2. Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.

import roman
from datetime import datetime


# 2. Real-time data of Peru's GDP growth rates
#-------------------------------------------------------------------------------------------------------------------------------

import psycopg2  # For interacting with PostgreSQL databases
from sqlalchemy import create_engine, text  # For creating and executing SQL queries using SQLAlchemy


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following code lines will create folders in your current path, call them to import and export your outputs. <div/>

# Data clean-up

In [ ]:
from gdp_revisions_datasets_functions import *

# Exclusivas para tablas entregadas por el BCRP

In [ ]:
# 1.

In [ ]:
def limpiar_nombres_columnas(df):
    # Eliminar tildes y convertir a minúsculas los nombres de las columnas
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    return df

In [ ]:
# 2.

In [ ]:
def ajustar_nombres_columnas(df):
    # Comprobar que la primera observación de la primera columna sea NaN
    if pd.isna(df.iloc[0, 0]) and pd.isna(df.iloc[0, -1]):
        # Verificar los nombres de las columnas
        if "sectores economicos" in df.columns[0] and "economic sectors" in df.columns[-1]:
            # Reemplazar NaN por los nombres de las columnas correspondientes
            df.iloc[0, 0] = "sectores economicos"
            df.iloc[0, -1] = "economic sectors"
    return df

In [ ]:
# 3.

In [ ]:
def rounding_values(df, decimales=1):
    # Iterar sobre todas las columnas del DataFrame
    for col in df.columns:
        # Verificar si la columna es de tipo float
        if df[col].dtype == 'float64':
            # Redondear los valores de la columna al número de decimales especificado
            df[col] = df[col].round(decimales)
    return df

# Importando csv

In [ ]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2\2010\ns-07-2010.csv'

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df.head(5)

# Test

In [ ]:


# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Diccionario para almacenar los DataFrames generados
old_dataframes_dict_1 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/old_carpetas_procesadas_1.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_archivo_csv(csv_path, engine):
    old_tables_dict_1 = {}  # Diccionario local para cada archivo CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    nombre_dataframe = f"{new_filename}_{table_counter}"
    old_tables_dict_1[nombre_dataframe] = df.copy()

    # Aplicar las funciones de limpieza a una copia del DataFrame
    df_clean = df.copy()

    # Funciones de limpieza (ajustar según sea necesario)
    df_clean = limpiar_nombres_columnas(df_clean)
    df_clean = ajustar_nombres_columnas(df_clean)
    df_clean = drop_rare_caracter_row(df_clean)
    df_clean = drop_nan_rows(df_clean)
    df_clean = drop_nan_columns(df_clean)
    df_clean = reset_index(df_clean)
    df_clean = remove_digit_slash(df_clean)
    df_clean = replace_var_perc_first_column(df_clean)
    df_clean = replace_var_perc_last_columns(df_clean)
    df_clean = replace_number_moving_average(df_clean)
    df_clean = relocate_last_column(df_clean)
    df_clean = clean_first_row(df_clean)
    df_clean = find_year_column(df_clean)
    year_columns = extract_years(df_clean)
    df_clean = get_months_sublist_list(df_clean, year_columns)
    df_clean = first_row_columns(df_clean)
    df_clean = clean_columns_values(df_clean)
    df_clean = convert_float(df_clean)
    df_clean = replace_set_sep(df_clean)
    df_clean = spaces_se_es(df_clean)
    df_clean = replace_services(df_clean)
    df_clean = rounding_values(df_clean, decimales=1)

    # Añadir la columna 'year' al DataFrame limpio
    df_clean.insert(0, 'year', year)
    
    # Añadir la columna 'id_ns' al DataFrame limpio
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Obtener la fecha correspondiente de la base de datos
    fecha = obtener_fecha(df_clean, engine)
    if fecha:
        # Añadir la columna 'date' al DataFrame limpio
        df_clean.insert(2, 'date', fecha)
    else:
        print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
    
    # Almacenar DataFrame limpio en old_dataframes_dict_1
    old_dataframes_dict_1[nombre_dataframe] = df_clean

    return id_ns, year, old_tables_dict_1

def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    csv_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.csv')]

    num_csv_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    old_tables_dict_1 = {}  # Declarar old_tables_dict_1 fuera del bucle principal
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = procesar_archivo_csv(csv_file, engine)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{table_counter}"
                
                # Almacenar DataFrame sin procesar en old_tables_dict_1
                old_tables_dict_1[nombre_df] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Aplicar las funciones de limpieza a una copia del DataFrame
                df_clean = limpiar_nombres_columnas(df_clean)
                df_clean = ajustar_nombres_columnas(df_clean)
                df_clean = drop_rare_caracter_row(df_clean)
                df_clean = drop_nan_rows(df_clean)
                df_clean = drop_nan_columns(df_clean)
                df_clean = reset_index(df_clean)
                df_clean = remove_digit_slash(df_clean)
                df_clean = replace_var_perc_first_column(df_clean)
                df_clean = replace_var_perc_last_columns(df_clean)
                df_clean = replace_number_moving_average(df_clean)
                df_clean = relocate_last_column(df_clean)
                df_clean = clean_first_row(df_clean)
                df_clean = find_year_column(df_clean)
                year_columns = extract_years(df_clean)
                df_clean = get_months_sublist_list(df_clean, year_columns)
                df_clean = first_row_columns(df_clean)
                df_clean = clean_columns_values(df_clean)
                df_clean = convert_float(df_clean)
                df_clean = replace_set_sep(df_clean)
                df_clean = spaces_se_es(df_clean)
                df_clean = replace_services(df_clean)
                df_clean = rounding_values(df_clean, decimales=1)
                
                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
                
                # Almacenar DataFrame limpio en old_dataframes_dict_1
                old_dataframes_dict_1[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {csv_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
        
        num_csv_procesados += 1  # Incrementar el número de CSVs procesados por cada archivo en la carpeta

    return num_csv_procesados, num_dataframes_generados, old_tables_dict_1

def procesar_carpetas():
    base_folder = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1'
    carpetas = [os.path.join(base_folder, d) for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_1 = {}  # Inicializar old_tables_dict_1 aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_csv_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar old_tables_dict_1 con los valores devueltos de procesar_carpeta()
        old_tables_dict_1.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_csv_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return old_tables_dict_1  # Devolver old_tables_dict_1 al final de la función

if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    old_tables_dict_1 = procesar_carpetas()  # Capturar el valor devuelto de procesar_carpetas()


In [ ]:
old_tables_dict_1['ns_03_2010_1']

In [ ]:
old_dataframes_dict_1['ns_03_2010_1']

# TABLA 1

# Cargando desde excel

In [ ]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1\table_1\00_1.csv'

# Carrega o arquivo CSV para um DataFrame
df_clean = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df_clean

# Total by year

In [ ]:
def replace_total_with_year(df):
    # Reemplazar 'TOTAL' con 'AÑO' en la primera fila
    df.iloc[0] = df.iloc[0].apply(lambda x: 'AÑO' if "TOTAL" in str(x) else x)
    return df

In [ ]:
df_clean = replace_total_with_year(df_clean)
df_clean

# Test

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
from tkinter import Tk, messagebox, TOP, YES, NO
from sqlalchemy import create_engine

# Establecer la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Diccionario para almacenar los DataFrames generados
old_dataframes_dict_2 = {}

# Ruta del archivo de registro de carpetas procesadas
registro_path = 'dataframes_record/old_carpetas_procesadas_2.txt'

# Función para corregir los nombres de los meses
def corregir_nombre_mes(mes):
    meses_mapping = {
        'setiembre': 'septiembre',
        # Agrega más mapeos si es necesario para otros nombres de meses
    }
    return meses_mapping.get(mes, mes)

def registrar_carpeta_procesada(carpeta, num_archivos_procesados):
    with open(registro_path, 'a') as file:
        file.write(f"{carpeta}:{num_archivos_procesados}\n")

def carpeta_procesada(carpeta):
    if not os.path.exists(registro_path):
        return False
    with open(registro_path, 'r') as file:
        for line in file:
            if line.startswith(carpeta):
                return True
    return False

def obtener_fecha(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    fecha = pd.read_sql(query, engine)
    return fecha.iloc[0, 0] if not fecha.empty else None

def procesar_archivo_csv(csv_path, engine):
    old_tables_dict_2 = {}  # Diccionario local para cada archivo CSV
    table_counter = 1

    filename = os.path.basename(csv_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No se encontraron coincidencias para id_ns y year en el nombre del archivo:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(csv_path))[0].replace('-', '_')

    df = pd.read_csv(csv_path, delimiter=';')
    
    nombre_dataframe = f"{new_filename}_{table_counter}"
    old_tables_dict_2[nombre_dataframe] = df.copy()

    # Aplicar las funciones de limpieza a una copia del DataFrame
    df_clean = df.copy()

    # Funciones de limpieza (ajustar según sea necesario)
    df_clean = replace_total_with_year(df_clean)
    df_clean = drop_nan_rows(df_clean)
    year_columns = extract_years(df_clean)
    df_clean = roman_arabic(df_clean)
    df_clean = fix_duplicates(df_clean)
    df_clean = relocate_last_column(df_clean)
    df_clean = replace_first_row_nan(df_clean)
    df_clean = clean_first_row(df_clean)
    df_clean = get_quarters_sublist_list(df_clean, year_columns)
    df_clean = reset_index(df_clean)
    df_clean = first_row_columns(df_clean)
    df_clean = clean_columns_values(df_clean)
    df_clean = reset_index(df_clean)
    df_clean = convert_float(df_clean)
    df_clean = replace_set_sep(df_clean)
    df_clean = spaces_se_es(df_clean)
    df_clean = replace_services(df_clean)
    df_clean = rounding_values(df_clean, decimales=1)

    # Añadir la columna 'year' al DataFrame limpio
    df_clean.insert(0, 'year', year)
    
    # Añadir la columna 'id_ns' al DataFrame limpio
    df_clean.insert(1, 'id_ns', id_ns)
    
    # Obtener la fecha correspondiente de la base de datos
    fecha = obtener_fecha(df_clean, engine)
    if fecha:
        # Añadir la columna 'date' al DataFrame limpio
        df_clean.insert(2, 'date', fecha)
    else:
        print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
    
    # Almacenar DataFrame limpio en old_dataframes_dict_2
    old_dataframes_dict_2[nombre_dataframe] = df_clean

    return id_ns, year, old_tables_dict_2

def procesar_carpeta(carpeta, engine):
    print(f"Procesando la carpeta {os.path.basename(carpeta)}")
    csv_files = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith('.csv')]

    num_csv_procesados = 0
    num_dataframes_generados = 0

    table_counter = 1  # Inicializar el contador de tabla aquí
    old_tables_dict_2 = {}  # Declarar old_tables_dict_2 fuera del bucle principal
    
    for csv_file in csv_files:
        id_ns, year, tables_dict_temp = procesar_archivo_csv(csv_file, engine)

        if tables_dict_temp:
            for nombre_df, df in tables_dict_temp.items():
                nombre_archivo = os.path.splitext(os.path.basename(csv_file))[0].replace('-', '_')
                nombre_df = f"{nombre_archivo}_{table_counter}"
                
                # Almacenar DataFrame sin procesar en old_tables_dict_2
                old_tables_dict_2[nombre_df] = df.copy()
                
                # Procesar y limpiar el DataFrame
                df_clean = df.copy()
                
                # Aplicar las funciones de limpieza a una copia del DataFrame
                df_clean = replace_total_with_year(df_clean)
                df_clean = drop_nan_rows(df_clean)
                year_columns = extract_years(df_clean)
                df_clean = roman_arabic(df_clean)
                df_clean = fix_duplicates(df_clean)
                df_clean = relocate_last_column(df_clean)
                df_clean = replace_first_row_nan(df_clean)
                df_clean = clean_first_row(df_clean)
                df_clean = get_quarters_sublist_list(df_clean, year_columns)
                df_clean = reset_index(df_clean)
                df_clean = first_row_columns(df_clean)
                df_clean = clean_columns_values(df_clean)
                df_clean = reset_index(df_clean)
                df_clean = convert_float(df_clean)
                df_clean = replace_set_sep(df_clean)
                df_clean = spaces_se_es(df_clean)
                df_clean = replace_services(df_clean)
                df_clean = rounding_values(df_clean, decimales=1)
                
                # Añadir la columna 'year' al DataFrame limpio
                df_clean.insert(0, 'year', year)
                
                # Añadir la columna 'id_ns' al DataFrame limpio
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Obtener la fecha correspondiente de la base de datos
                fecha = obtener_fecha(df_clean, engine)
                if fecha:
                    # Añadir la columna 'date' al DataFrame limpio
                    df_clean.insert(2, 'date', fecha)
                else:
                    print("No se encontró fecha en la base de datos para id_ns:", id_ns, "y year:", year)
                
                # Almacenar DataFrame limpio en old_dataframes_dict_2
                old_dataframes_dict_2[nombre_df] = df_clean

                print(f'  {table_counter}. El dataframe generado para el archivo {csv_file} es: {nombre_df}')
                num_dataframes_generados += 1
                table_counter += 1  # Incrementar el contador de tabla aquí
        
        num_csv_procesados += 1  # Incrementar el número de CSVs procesados por cada archivo en la carpeta

    return num_csv_procesados, num_dataframes_generados, old_tables_dict_2

def procesar_carpetas():
    base_folder = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2'
    carpetas = [os.path.join(base_folder, d) for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d)) and re.match(r'\d{4}', d)]
    
    old_tables_dict_2 = {}  # Inicializar old_tables_dict_2 aquí
    
    for carpeta in carpetas:
        if carpeta_procesada(carpeta):
            print(f"La carpeta {carpeta} ya ha sido procesada.")
            continue
        
        num_csv_procesados, num_dataframes_generados, tables_dict_temp = procesar_carpeta(carpeta, engine)
        
        # Actualizar old_tables_dict_2 con los valores devueltos de procesar_carpeta()
        old_tables_dict_2.update(tables_dict_temp)
        
        registrar_carpeta_procesada(carpeta, num_csv_procesados)

        # Preguntar al usuario si desea continuar con la siguiente carpeta
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Para asegurar que la ventana esté en primer plano
        
        mensaje = f"Se han generado {num_dataframes_generados} dataframes en la carpeta {carpeta}. ¿Deseas continuar con la siguiente carpeta?"
        continuar = messagebox.askyesno("Continuar", mensaje)
        root.destroy()

        if not continuar:
            print("Procesamiento detenido por el usuario.")
            break  # Romper el bucle for si el usuario decide no continuar

    print("Procesamiento completado para todas las carpetas.")  # Add a message to indicate completion

    return old_tables_dict_2  # Devolver old_tables_dict_2 al final de la función
    
if __name__ == "__main__":
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')
    password = os.environ.get('CIUP_SQL_PASS')
    host = os.environ.get('CIUP_SQL_HOST')
    port = 5432
    database = 'gdp_revisions_datasets'

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    old_tables_dict_2 = procesar_carpetas()  # Capturar el valor devuelto de procesar_carpetas()

In [ ]:
old_tables_dict_2['ns_04_2010_1']

In [ ]:
old_dataframes_dict_2['ns_32_2010_1']

# TO SQL

In [ ]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\to_sql\old_raw_data_delivered.csv'

# Carrega o arquivo CSV para um DataFrame
old_raw_data_delivered = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
old_raw_data_delivered.head(5)

# Duplicate table 1

In [ ]:
import os
import shutil
import psycopg2
import pandas as pd

# Obtener variables de entorno
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Verificar si todas las variables de entorno están definidas
if not all([host, user, password]):
    raise ValueError("Faltan algunas variables de entorno (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Crear la cadena de conexión
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Definir la consulta SQL para importar datos
query = f"SELECT * FROM old_raw_data_delivered"

# Importar los datos a un DataFrame de pandas
df = pd.read_sql(query, engine)

# Ruta base donde están las carpetas de años
base_path = r"C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 1"

# Función para duplicar archivos
def duplicate_files(year, df_year, base_path):
    year_path = os.path.join(base_path, str(year))
    files = sorted([f for f in os.listdir(year_path) if f.endswith('.csv')])

    # Obtener los archivos existentes
    existing_files = {int(f.split('-')[1]): f for f in files}
    
    # Obtener el último archivo del año anterior
    if year > 1994:
        prev_year_path = os.path.join(base_path, str(year - 1))
        prev_files = sorted([f for f in os.listdir(prev_year_path) if f.endswith('.csv')])
        if prev_files:
            last_prev_file = prev_files[-1]
            last_prev_id_ns = int(last_prev_file.split('-')[1])
        else:
            last_prev_file = None
            last_prev_id_ns = None
    else:
        last_prev_file = None
        last_prev_id_ns = None

    # Inicializar variables para duplicación
    last_existing_file = last_prev_file
    last_existing_id_ns = last_prev_id_ns

    for index, row in df_year.iterrows():
        id_ns = row['id_ns']
        if row['delivered_1'] == 1:
            last_existing_file = existing_files[id_ns]
            last_existing_id_ns = id_ns
        else:
            # Crear nombre del nuevo archivo duplicado
            new_file_name = f"ns-{id_ns:02d}-{year}.csv"
            new_file_path = os.path.join(year_path, new_file_name)

            # Duplicar archivo
            if last_existing_file:
                if last_existing_file in existing_files.values():
                    src_file_path = os.path.join(year_path, last_existing_file)
                else:
                    src_file_path = os.path.join(prev_year_path, last_existing_file)
                shutil.copy(src_file_path, new_file_path)
                print(f"Duplicated {last_existing_file} to {new_file_name}")
            else:
                print(f"No existing file to duplicate for {new_file_name}")

# Procesar cada año
for year in range(2010, 2013):
    df_year = df[df['year'] == year]
    duplicate_files(year, df_year, base_path)


# Duplicate table 2

In [ ]:
import os
import shutil
import psycopg2
import pandas as pd

# Obtener variables de entorno
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Verificar si todas las variables de entorno están definidas
if not all([host, user, password]):
    raise ValueError("Faltan algunas variables de entorno (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Crear la cadena de conexión
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Definir la consulta SQL para importar datos
query = f"SELECT * FROM old_raw_data_delivered"

# Importar los datos a un DataFrame de pandas
df = pd.read_sql(query, engine)

# Ruta base donde están las carpetas de años
base_path = r"C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\tabla 2"

# Función para duplicar archivos
def duplicate_files(year, df_year, base_path):
    year_path = os.path.join(base_path, str(year))
    files = sorted([f for f in os.listdir(year_path) if f.endswith('.csv')])

    # Obtener los archivos existentes
    existing_files = {int(f.split('-')[1]): f for f in files}
    
    # Obtener el último archivo del año anterior
    if year > 1994:
        prev_year_path = os.path.join(base_path, str(year - 1))
        prev_files = sorted([f for f in os.listdir(prev_year_path) if f.endswith('.csv')])
        if prev_files:
            last_prev_file = prev_files[-1]
            last_prev_id_ns = int(last_prev_file.split('-')[1])
        else:
            last_prev_file = None
            last_prev_id_ns = None
    else:
        last_prev_file = None
        last_prev_id_ns = None

    # Inicializar variables para duplicación
    last_existing_file = last_prev_file
    last_existing_id_ns = last_prev_id_ns

    for index, row in df_year.iterrows():
        id_ns = row['id_ns']
        if row['delivered_2'] == 1:
            last_existing_file = existing_files[id_ns]
            last_existing_id_ns = id_ns
        else:
            # Crear nombre del nuevo archivo duplicado
            new_file_name = f"ns-{id_ns:02d}-{year}.csv"
            new_file_path = os.path.join(year_path, new_file_name)

            # Duplicar archivo
            if last_existing_file:
                if last_existing_file in existing_files.values():
                    src_file_path = os.path.join(year_path, last_existing_file)
                else:
                    src_file_path = os.path.join(prev_year_path, last_existing_file)
                shutil.copy(src_file_path, new_file_path)
                print(f"Duplicated {last_existing_file} to {new_file_name}")
            else:
                print(f"No existing file to duplicate for {new_file_name}")

# Procesar cada año
for year in range(2010, 2013):
    df_year = df[df['year'] == year]
    duplicate_files(year, df_year, base_path)
